# CS-7643 Deep Learning Final Project

## Data Preprocessing

#### by Zhiyin (Steven) Lu and Yixuan (Elliot) Xie

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

# %cd drive/MyDrive/cs7643_final_project
# %ls

In [ ]:
import glob
import pandas as pd

In [ ]:
# get scan and mask image information and organize into a dataframe
scan_dir = './data/scans/train/**/*.png'
mask_dir = './data/masks/train/**/*.npy'
save_dir = './data/data.csv'
save_debug_dir = './data/debug.csv'

scan_path = sorted(glob.glob(scan_dir, recursive=True))
mask_path = sorted(glob.glob(mask_dir, recursive=True))
data = pd.DataFrame(data={'scan_path': scan_path, 'mask_path': mask_path})

# extract information into separate columns
data['id'] = data.scan_path.map( lambda x: x.split('/')[-3] + '_' + '_'.join(x.split('/')[-1].split('_')[:2]) )
data['case'] = data.id.map( lambda x: int(x.split('_')[0][4:]) )
data['day'] = data.id.map( lambda x: int(x.split('_')[1][3:]) )
data['slice'] = data.id.map( lambda x: int(x.split('_')[-1]) )
data['height'] = data.scan_path.map( lambda x: int(x.split('/')[-1].split('_')[2]) )
data['width'] = data.scan_path.map( lambda x: int(x.split('/')[-1].split('_')[3]) )

# get the segmentation information from train.csv and organize into a 'masks' dataframe
train_csv_dir = './data/train.csv'
train_csv = pd.read_csv(train_csv_dir)
train_csv.segmentation = train_csv.segmentation.fillna('')
train_csv['rle_len'] = train_csv.segmentation.map(len)

# group segmentation into lists and calculate the length of the segmentation list
seg = train_csv.groupby(['id'])['segmentation'].agg(list).to_frame()
rle_len = train_csv.groupby(['id'])['rle_len'].agg(sum).to_frame()

# merge and add segmentation list and rle_length to the 'mask_info' dataframe
mask_info = seg.merge(rle_len, on=['id'])
mask_info['empty'] = (mask_info['rle_len'] == 0)

# merge 'scans' and 'masks' into one dataframe based on IDs
data = data.merge(mask_info, on=['id'])

# Total: 38496
display(data)

data.to_csv(save_dir, index=False)

In [ ]:
##### RANDOM SAMPLING TO GET A SMALL DATASET FOR DEBUG PURPOSE #####
debug = data.sample(n=20, random_state=1)

display(debug)

debug.to_csv(save_debug_dir, index=False)